In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *
from PG_algorithms import *
from ActorCritic import *
#np.set_printoptions(precision=4)
import math
from utils import *

nrows = 20
nS = nrows**2
nA = 4
seed = get_current_seed()
gamma = .9
tau = 1.
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = FrozenLakeEnv(is_slippery=False, seed=seed, desc=generate_random_map(nrows), r_shape=True)#, render_mode="human")
xi_frozen = np.ones(env.nS) * 1/env.nS
tmdp = TMDP(env, xi_frozen, tau=tau, gamma=gamma, seed=seed)
tmdp.reset()

Current seed for result reproducibility: 198857940169149813041072980569695548434


(0, {'prob': 1})

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
actor_func = ActorNet(nS, nA, hidden_dim=nS).to(device)
value_func = ValueNet(nS, hidden_dim=nS).to(device)
q_func = QNet(nS, nA, hidden_dim=nS+nA).to(device)

In [4]:
tmdp.update_tau(1.)
loss_param = list(actor_func.parameters()) + list(value_func.parameters())
loss_opt = torch.optim.AdamW(loss_param, lr=1e-3)
q_opt = torch.optim.AdamW(q_func.parameters(), lr=1e-3)

cur_res = curriculum_PPO(tmdp, actor_func, value_func, q_func, loss_opt, q_opt, alpha=.2, eps=.2, vf_coeff=.5, h_coeff=.3, episodes=1000000, batch_nS=1, biased=False)


[(0.0, 1.0)]
[0.0]
Updating tau with tau_star:  1.0
Running episode 2 reward -3.0
[(0.0, 1.0)]
[0.0]
Updating tau with tau_star:  1.0
Running episode 3 reward -1.0
[(0.0, 1.0)]
[0.0]
Updating tau with tau_star:  1.0
Running episode 6 reward -3.0
[(0.00433, 1.0), (0.0, 1.0)]
[2.874354800516691e-06, 0.0]
Running episode 9 reward -3.0
[(0.013, 1.0), (0.0, 0.99999)]
[1.7592225966168642e-05, 1.711894999137413e-09]
Running episode 10 reward -1.0
[(0.00329, 1.0), (0.0, 0.99999)]
[1.4881213256710023e-06, 7.054141177100808e-09]
Running episode 16 reward -6.0
[(0.0, 0.99998)]
[1.372827710715758e-08]
Updating tau with tau_star:  0.99998
Running episode 22 reward -6.0
[(0.0, 0.99997)]
[1.2528096872656249e-08]
Updating tau with tau_star:  0.99997
Running episode 29 reward -7.0
[(0.0, 0.99995)]
[2.4771615542373743e-08]
Updating tau with tau_star:  0.99995
Running episode 33 reward -4.0
[(0.0, 0.99993)]
[3.407432850364938e-08]
Updating tau with tau_star:  0.99993
Running episode 34 reward -1.0
[(0.0,

In [ ]:

pi = cur_res["policy_pi"].get_probabilities()
tmdp = TMDP(env, xi_frozen, tau=0., gamma=gamma, seed=seed)
env.render_mode = "human"
tmdp.reset()
done = False
step = 0
while True:
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break